In [13]:
#import dependencies for Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [14]:
#Set the executable path
executable_path = {'executable_path': ChromeDriverManager().install()}

#set up the Browser url for scrapping
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\owner\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [15]:
#assign the URL and instruct the browser to visit it
#visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)

#operational deal for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
#what we are doing here is 1. searching for elements with a specific comination of tag (div and sttribute list_text)
# and 2. we are telling the browser to wait one second before searching for components

True

In [16]:
# Set up the html parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')
#slide_elem looks for the <div /> tag and its descendent
#this element holds all other elements in it
# the . is used for selecting classes (aka list_text here in this example)
# css works from right to left (such as returning the last item instead of the first)
# because of this, when using select_one, the first matching element returned will be a <li /> element with a class of slide

In [17]:
#assign the title and summary text to variables
#begin our scraping
slide_elem.find('div', class_='content_title')
#look for specific data

<div class="content_title">NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet</div>

In [18]:
# use the parent element to find the first 'a' tag and save it as 'news_title'
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title
# get text removes the html code and only keeps the text in between
#.find() is used when we want only the first class and attribute we specified
#.find_all() is used when we want to retrieve all of the tags and attributes

'NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet'

In [19]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. "

### Featured Images

In [20]:
# set up the url 
#Visit the space image site
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [21]:
#find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [22]:
# set up the html.parser to full parse the image so we can scrape the full size
html = browser.html
img_soup = soup(html, 'html.parser')

In [23]:
# find the relative image URL 
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel
# an img tag is nested within the HTML, so we have included it
#.get('src') pulls the link to the image

'image/featured/mars2.jpg'

In [24]:
# use the base URL to create an absolute URL
img_url = f'https://speaceimages-mars.com{img_url_rel}'
img_url

'https://speaceimages-mars.comimage/featured/mars2.jpg'

### Getting the table data from mars facts

In [25]:
#create a new datafram from the HTML table
df = pd.read_html('https://galaxyfacts-mars.com')[0]
#assign columns to the datafram for additional clarity
df.columns=['description', 'Mars', 'Earth']
# turn the description column into the dataframes index
#inplace=true means that the updated index will remain in place without having to reassign the df to a new variable
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [26]:
# use pandas to revert the dataframe back into HTML usinng the .to_html() function
df.to_html()
#this creates a table element with a lot of nested elements inside

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [27]:
# end the automated browsing session
browser.quit()